In [1]:
from transformers import AutoModelForTokenClassification, AutoTokenizer,DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
import evaluate  
from datasets import load_dataset
import numpy as np

d:\anaconda\envs\deepstudy\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = load_dataset('doushabao4766/msra_ner_k_V3')
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 45001
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 3443
    })
})

In [3]:
for row in ds['train']:
    print(row['tokens'])
    print(row['ner_tags'])
    
    break

['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [4]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')

## 实体映射字典
'O':0
'B-PER':1
'I-PER':2
'B-LOC':3
'I-LOC':4
'B-ORG':5
'I-ORG':6

In [5]:
entites = ['O'] + list({'PER', "LOC", "ORG"})
entity_index = {entity:i for i, entity in enumerate(entites)}
tags = ['O']
for entity in entites[1:]:
    tags.append('B-' + entity)
    tags.append('I-' + entity)

In [6]:
entity_index

{'O': 0, 'PER': 1, 'LOC': 2, 'ORG': 3}

In [7]:
tags

['O', 'B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG']

In [8]:
def data_input_proc(item):
    # 文本已经分为字符，且tag索引也已经提供
    # 所以数据预处理反而简单
    # 导入已拆分为字符的文本列表，需要设置参数is_split_into_words=True
    input_data = tokenizer(item['tokens'], 
                           truncation=True,
                           add_special_tokens=False, 
                           max_length=512, 
                           is_split_into_words=True)
    
    labels = [lbl[:512] for lbl in item['ner_tags']]
    input_data['labels'] = labels
    return input_data

ds1 = ds.map(data_input_proc, batched=True)  # batched 每次传入自定义方法样本数量多个

In [9]:
ds1.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [10]:
args = TrainingArguments(
    output_dir="ner_train",  # 模型训练工作目录（tensorboard，临时模型存盘文件，日志）
    num_train_epochs = 3,    # 训练 epoch
    # save_safetensors=False,  # 设置False保存文件可以通过torch.load加载
    per_device_train_batch_size=4,  # 训练批次
    per_device_eval_batch_size=4,
    report_to='tensorboard',  # 训练输出记录
    eval_strategy="epoch",
)

In [11]:
tags

['O', 'B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG']

In [12]:
id2lb = {i:tag for i,tag in enumerate(tags)}
lb2id = {tag:i for i,tag in enumerate(tags)}
model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese', num_labels=7, id2label=id2lb, label2id=lb2id)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [13]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [14]:
# metric 方法
def compute_metric(result):
    # result 是一个tuple (predicts, labels)
    
    # 获取评估对象
    seqeval = evaluate.load('seqeval')
    predicts,labels = result
    predicts = np.argmax(predicts, axis=2)
    
    # 准备评估数据
    predicts = [[tags[p] for p,l in zip(ps,ls) if l != -100]
                 for ps,ls in zip(predicts,labels)]
    labels = [[tags[l] for p,l in zip(ps,ls) if l != -100]
                 for ps,ls in zip(predicts,labels)]
    results = seqeval.compute(predictions=predicts, references=labels)

    return results

In [15]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, padding=True)

trainer = Trainer(
    model,
    args,
    train_dataset=ds1['train'],
    eval_dataset=ds1['test'],
    data_collator=data_collator,
    compute_metrics=compute_metric
)

In [ ]:
trainer.train()

In [17]:
from transformers import pipeline

In [ ]:
pipeline = pipeline('token-classification', 'ner_train/checkpoint-2110',)
pipeline('双方确定了今后发展中美关系的指导方针')